In [25]:
import numpy as np
import matplotlib.image as mpimg
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from skimage.color import rgb2gray

In [26]:
import os
from os import listdir
from os.path import isfile, join

In [27]:
image_folder_path = "positive_red/"

onlyfiles = [f  for f in listdir(image_folder_path) if isfile(join(image_folder_path, f))]
full_filenames = [join(image_folder_path,this_image) for this_image in onlyfiles]
A  = []
for this_filename in full_filenames:
    #append(hog(resize(i, (128, 64)),orientations=8,pixels_per_cell=(8, 8),cells_per_block=(1, 1),channel_axis=-1,))
    #values = hog(rgb2gray(resize(mpimg.imread(this_filename)[:,:,:3],(128,64))), orientations=8,pixels_per_cell=(8, 8),cells_per_block=(1, 1), channel_axis= -1)
    values = hog(rgb2gray(resize(mpimg.imread(this_filename)[:,:,:3],(128,64))), orientations=8,pixels_per_cell=(8, 8),cells_per_block=(1, 1))
    A.append(values)


In [28]:
B = []
image_folder_path = "negative/"

onlyfiles = [f for f in listdir(image_folder_path) if isfile(join(image_folder_path, f))]

full_filenames = [join(image_folder_path,this_image) for this_image in onlyfiles]


for this_filename in full_filenames:
    #values = hog(resize(mpimg.imread(this_filename)[:,:,:3],(128,64)), orientations=8,pixels_per_cell=(8, 8),cells_per_block=(1, 1),channel_axis=-1)
    values = hog(rgb2gray(resize(mpimg.imread(this_filename)[:,:,:3],(128,64))), orientations=8,pixels_per_cell=(8, 8),cells_per_block=(1, 1))
    B.append(values)


In [29]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [30]:
X = np.array(A + B)
Y = np.array([1 for i in A] + [0 for i in B])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [32]:
from sklearn.model_selection import GridSearchCV

ml = svm.SVC() 

param_grid = {'C': [ 0.0005, 0.001,0.005,0.01,0.05,0.1,0.5, 1, 2,3,4,5,6,7,8,9,10,11,12,13,15, 100, 1000,10000], 
              'gamma': [1,0.5,0.1,0.05,0.01,0.005,0.001,0.0005,0.0001,0.00005,0.00001,0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(ml, param_grid, refit = True, verbose = 1,cv=15)

grid_search=grid.fit(X, Y)

Fitting 15 folds for each of 288 candidates, totalling 4320 fits


In [33]:
model = grid.best_estimator_
model.fit(X, Y)

SVC(C=3, gamma=0.05)

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
accuracy_score(Y,model.predict(X))

1.0

In [36]:
np.savetxt("support_vectors_r.txt", model.support_vectors_)
np.savetxt("model.dual_coef_r.txt", model.dual_coef_)

In [37]:
model.intercept_

array([-0.91069488])

In [668]:
support = np.loadtxt('support_vectors_r.txt')

In [669]:
model.support_vectors_.shape

(204, 1024)

In [670]:
support.shape

(177, 1024)

In [671]:
def RBF(X,Y, gamma):
    
    # Free parameter gamma
    if gamma == None:
        gamma = 1.0/X.shape[1]
        
    # RBF kernel Equation
    K = np.exp(-gamma * np.sum((X - Y)**2, axis = -1))
    
    return K

In [672]:
def dfk(coef,Xex,vectors,g,bet):
    k = np.sign(np.sum(coef*RBF(Xex,vectors,g)) + bet)
    return k

In [673]:
dfk(model.dual_coef_,X[100],support,model.gamma,model.intercept_) 

ValueError: operands could not be broadcast together with shapes (1,204) (177,) 